In [1]:
import pygame
import sys
import numpy as np
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os
import math
from typing import Optional, List, Tuple
import json
from datetime import datetime

# Initialisation de Pygame
pygame.init()

# Constantes
WINDOW_WIDTH = 1200
WINDOW_HEIGHT = 800
BOARD_SIZE = 400
CELL_SIZE = BOARD_SIZE // 3
BOARD_OFFSET_X = (WINDOW_WIDTH - BOARD_SIZE) // 2  # Centrer horizontalement
BOARD_OFFSET_Y = 150  # Ajusté pour scores en haut et résultat en bas

# Couleurs
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (100, 149, 237)
RED = (220, 20, 60)
GREEN = (34, 139, 34)
GRAY = (128, 128, 128)
LIGHT_GRAY = (211, 211, 211)
DARK_GRAY = (64, 64, 64)
GOLD = (255, 215, 0)
SILVER = (192, 192, 192)
BACKGROUND = (240, 248, 255)
BUTTON_COLOR = (70, 130, 180)
BUTTON_HOVER = (100, 149, 237)

# Constantes de jeu
VIDE = 0
JOUEUR_HUMAIN = 1  # X
JOUEUR_IA = 2      # O

class MorpionIAEngine:
    """Moteur IA pour le jeu de Morpion"""
    
    def __init__(self):
        self.plateau = [0] * 9
        self.modele = None
        self.scaler = None
        self.modele_charge = False
        
        # Paramètres stratégiques
        self.priorite_gagner = 1000
        self.priorite_bloquer = 900
        self.priorite_menace = 100
        
    def charger_modele(self, chemin_modele: str, chemin_scaler: str = None) -> bool:
        """Charge le modèle ML et le scaler"""
        if not os.path.exists(chemin_modele):
            print(f"❌ Fichier modèle non trouvé: {chemin_modele}")
            return False
            
        if chemin_scaler is None:
            chemin_scaler = chemin_modele.replace(".pkl", "_scaler.pkl")
        
        try:
            self.modele = joblib.load(chemin_modele)
            if os.path.exists(chemin_scaler):
                self.scaler = joblib.load(chemin_scaler)
            self.modele_charge = True
            print(f"✅ Modèle chargé: {chemin_modele}")
            return True
        except Exception as e:
            print(f"❌ Erreur chargement modèle: {e}")
            return False
    
    def detecter_coup_gagnant(self, plateau: List[int], joueur: int) -> Optional[int]:
        """Détecte s'il y a un coup gagnant immédiat"""
        for i in range(9):
            if plateau[i] == 0:
                plateau_temp = plateau.copy()
                plateau_temp[i] = joueur
                if self.verifier_victoire_plateau(plateau_temp, joueur):
                    return i
        return None
    
    def detecter_coup_bloquant(self, plateau: List[int], joueur_adverse: int) -> Optional[int]:
        """Détecte s'il faut bloquer un coup gagnant adverse"""
        return self.detecter_coup_gagnant(plateau, joueur_adverse)
    
    def verifier_victoire_plateau(self, plateau: List[int], joueur: int) -> bool:
        """Vérifie si le joueur a gagné"""
        for i in range(3):
            if all(plateau[i*3 + j] == joueur for j in range(3)):
                return True
        for j in range(3):
            if all(plateau[i*3 + j] == joueur for i in range(3)):
                return True
        if all(plateau[i*3 + i] == joueur for i in range(3)):
            return True
        if all(plateau[i*3 + (2-i)] == joueur for i in range(3)):
            return True
        return False
    
    def compter_menaces(self, plateau: List[int], joueur: int) -> int:
        """Compte les menaces pour un joueur"""
        menaces = 0
        for i in range(3):
            ligne = [plateau[i*3 + j] for j in range(3)]
            if ligne.count(joueur) == 2 and ligne.count(0) == 1:
                menaces += 1
        for j in range(3):
            colonne = [plateau[i*3 + j] for i in range(3)]
            if colonne.count(joueur) == 2 and colonne.count(0) == 1:
                menaces += 1
        diag1 = [plateau[i*3 + i] for i in range(3)]
        if diag1.count(joueur) == 2 and diag1.count(0) == 1:
            menaces += 1
        diag2 = [plateau[i*3 + (2-i)] for i in range(3)]
        if diag2.count(joueur) == 2 and diag2.count(0) == 1:
            menaces += 1
        return menaces
    
    def creer_features_supplementaires(self, plateau: List[int], joueur_actuel: int) -> dict:
        """Crée les features pour le modèle ML"""
        features = {}
        for i in range(9):
            features[f'pos_{i}'] = plateau[i]
        features['joueur_actuel'] = joueur_actuel
        features['nb_x'] = plateau.count(1)
        features['nb_o'] = plateau.count(2)
        features['nb_vides'] = plateau.count(0)
        features['centre_occupe'] = 1 if plateau[4] != 0 else 0
        coins = [0, 2, 6, 8]
        features['nb_coins_occupes'] = sum(1 for pos in coins if plateau[pos] != 0)
        bords = [1, 3, 5, 7]
        features['nb_bords_occupes'] = sum(1 for pos in bords if plateau[pos] != 0)
        features['menaces_x'] = self.compter_menaces(plateau, 1)
        features['menaces_o'] = self.compter_menaces(plateau, 2)
        features['diag_principale'] = plateau[0] + plateau[4] + plateau[8]
        features['diag_secondaire'] = plateau[2] + plateau[4] + plateau[6]
        return features
    
    def obtenir_score_modele(self, plateau_temp: List[int], coup: int) -> float:
        """Obtient le score du modèle ML pour un coup"""
        if not self.modele_charge:
            return 0
        try:
            features = self.creer_features_supplementaires(plateau_temp, JOUEUR_IA)
            features_df = pd.DataFrame([features])
            if self.scaler is not None:
                features_scaled = self.scaler.transform(features_df)
                probabilites = self.modele.predict_proba(features_scaled)[0]
            else:
                probabilites = self.modele.predict_proba(features_df)[0]
            return probabilites[coup] if coup < len(probabilites) else 0
        except Exception as e:
            print(f"Erreur prédiction ML: {e}")
            return 0
    
    def predire_coup_ia(self, plateau: List[int]) -> Optional[int]:
        """Prédit le meilleur coup pour l'IA"""
        coups_possibles = [i for i in range(9) if plateau[i] == 0]
        if not coups_possibles:
            return None
        coup_gagnant = self.detecter_coup_gagnant(plateau, JOUEUR_IA)
        if coup_gagnant is not None:
            return coup_gagnant
        coup_bloquant = self.detecter_coup_bloquant(plateau, JOUEUR_HUMAIN)
        if coup_bloquant is not None:
            return coup_bloquant
        if self.modele_charge:
            return self.predire_avec_modele_et_strategie(plateau, coups_possibles)
        return self.coup_intelligent_fallback(coups_possibles)
    
    def predire_avec_modele_et_strategie(self, plateau: List[int], coups_possibles: List[int]) -> int:
        """Combine ML avec stratégie"""
        evaluations = []
        for coup in coups_possibles:
            plateau_temp = plateau.copy()
            plateau_temp[coup] = JOUEUR_IA
            score_ml = self.obtenir_score_modele(plateau_temp, coup)
            bonus = 0
            menaces_creees = self.compter_menaces(plateau_temp, JOUEUR_IA)
            bonus += menaces_creees * self.priorite_menace
            if coup == 4:
                bonus += 50
            elif coup in [0, 2, 6, 8]:
                bonus += 30
            score_total = score_ml + bonus
            evaluations.append((coup, score_total))
        return max(evaluations, key=lambda x: x[1])[0]
    
    def coup_intelligent_fallback(self, coups_possibles: List[int]) -> int:
        """Fallback intelligent"""
        if 4 in coups_possibles:
            return 4
        coins_dispo = [pos for pos in [0, 2, 6, 8] if pos in coups_possibles]
        if coins_dispo:
            return np.random.choice(coins_dispo)
        return np.random.choice(coups_possibles)

class Button:
    """Classe pour gérer les boutons"""
    
    def __init__(self, x: int, y: int, width: int, height: int, text: str, 
                 color: Tuple[int, int, int] = BUTTON_COLOR, 
                 hover_color: Tuple[int, int, int] = BUTTON_HOVER):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.hover_color = hover_color
        self.font = pygame.font.Font(None, 32)
        self.is_hovered = False
        self.is_enabled = True
    
    def handle_event(self, event: pygame.event.Event) -> bool:
        """Gère les événements du bouton"""
        if event.type == pygame.MOUSEMOTION:
            self.is_hovered = self.rect.collidepoint(event.pos)
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if self.rect.collidepoint(event.pos) and self.is_enabled:
                return True
        return False
    
    def draw(self, screen: pygame.Surface):
        """Dessine le bouton"""
        color = self.hover_color if self.is_hovered else self.color
        if not self.is_enabled:
            color = GRAY
        pygame.draw.rect(screen, color, self.rect)
        pygame.draw.rect(screen, BLACK, self.rect, 2)
        text_surface = self.font.render(self.text, True, WHITE)
        text_rect = text_surface.get_rect(center=self.rect.center)
        screen.blit(text_surface, text_rect)

class MorpionPygame:
    """Classe principale du jeu Morpion avec interface Pygame"""
    
    def __init__(self):
        self.screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
        pygame.display.set_caption("Morpion IA - Interface Pygame")
        
        self.ia_engine = MorpionIAEngine()
        self.plateau = [0] * 9
        self.tour_joueur = JOUEUR_HUMAIN
        self.partie_terminee = False
        self.message_status = "Votre tour! Cliquez sur une case."
        self.gagnant = None
        self.ligne_gagnante = None
        
        self.score_humain = 0
        self.score_ia = 0
        self.parties_nulles = 0
        self.historique_parties = []
        
        self.font_titre = pygame.font.Font(None, 48)
        self.font_normal = pygame.font.Font(None, 28)
        self.font_petit = pygame.font.Font(None, 24)
        self.font_grand = pygame.font.Font(None, 64)
        
        self.boutons = self.creer_boutons()
        
        self.animation_victoire = 0
        self.animation_coup_ia = 0
        self.derniere_position_ia = None
        
        self.modele_charge = False
        self.chemin_modele = ""
        
        self.clock = pygame.time.Clock()
        
        # Charger le modèle automatiquement
        self.charger_modele_interface()
    
    def creer_boutons(self) -> List[Button]:
        """Crée les boutons de l'interface"""
        boutons = []
        x_start = BOARD_OFFSET_X
        y_start = BOARD_OFFSET_Y + BOARD_SIZE + 80  # Sous le plateau
        
        boutons.append(Button(x_start, y_start, 200, 50, "Nouvelle Partie"))
        boutons.append(Button(x_start + 220, y_start, 200, 50, "Reset Scores"))
        
        return boutons
    
    def handle_events(self):
        """Gère les événements Pygame"""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            
            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                for bouton in self.boutons:
                    if bouton.handle_event(event):
                        if bouton.text == "Nouvelle Partie":
                            self.nouvelle_partie()
                        elif bouton.text == "Reset Scores":
                            self.reset_scores()
                if not self.partie_terminee and self.tour_joueur == JOUEUR_HUMAIN:
                    position = self.handle_click_plateau(pos)
                    if position is not None:
                        if self.jouer_coup(position):
                            self.draw()
                            pygame.time.wait(1500)
                            self.jouer_coup_ia()
    
    def update(self):
        """Met à jour l'état du jeu"""
        if self.animation_coup_ia > 0:
            self.animation_coup_ia -= 1
        if self.animation_victoire > 0:
            self.animation_victoire -= 1
    
    def draw(self):
        """Dessine tous les éléments à l'écran"""
        self.screen.fill(BACKGROUND)
        self.draw_scores()
        self.draw_plateau()
        self.draw_result()
        self.draw_ligne_gagnante()
        for bouton in self.boutons:
            bouton.draw(self.screen)
        pygame.display.flip()
    
    def run(self):
        """Boucle principale du jeu"""
        while True:
            self.handle_events()
            self.update()
            self.draw()
            self.clock.tick(60)
    
    def charger_modele_interface(self):
        """Charge le modèle par défaut"""
        chemin = "meilleur_modele_morpion.pkl"
        if self.ia_engine.charger_modele(chemin):
            self.modele_charge = True
            self.chemin_modele = chemin
            self.message_status = "Votre tour! Cliquez sur une case."
        else:
            self.message_status = "IA basique - Votre tour!"
    
    def handle_click_plateau(self, pos: Tuple[int, int]) -> Optional[int]:
        """Convertit un clic en position sur le plateau"""
        x, y = pos
        if (BOARD_OFFSET_X <= x <= BOARD_OFFSET_X + BOARD_SIZE and
            BOARD_OFFSET_Y <= y <= BOARD_OFFSET_Y + BOARD_SIZE):
            col = (x - BOARD_OFFSET_X) // CELL_SIZE
            row = (y - BOARD_OFFSET_Y) // CELL_SIZE
            if 0 <= row < 3 and 0 <= col < 3:
                return row * 3 + col
        return None
    
    def jouer_coup(self, position: int) -> bool:
        """Joue un coup pour le joueur humain"""
        if (self.partie_terminee or 
            self.plateau[position] != 0 or 
            self.tour_joueur != JOUEUR_HUMAIN):
            return False
        
        self.plateau[position] = JOUEUR_HUMAIN
        
        if self.verifier_victoire(JOUEUR_HUMAIN):
            self.terminer_partie("Vous avez gagné!", JOUEUR_HUMAIN)
            self.score_humain += 1
            return True
        elif sum(case != 0 for case in self.plateau) == 8:
            self.terminer_partie("Match nul!", None)
            self.parties_nulles += 1
            return True
        
        self.tour_joueur = JOUEUR_IA
        return True
    
    def jouer_coup_ia(self):
        """Joue un coup pour l'IA avec un délai de 2 secondes"""
        if self.partie_terminee or self.tour_joueur != JOUEUR_IA:
            return
        
        coup_ia = self.ia_engine.predire_coup_ia(self.plateau)
        if coup_ia is not None:
            self.plateau[coup_ia] = JOUEUR_IA
            self.derniere_position_ia = coup_ia
            self.animation_coup_ia = 30
            
            if self.verifier_victoire(JOUEUR_IA):
                self.terminer_partie("L'IA a gagné!", JOUEUR_IA)
                self.score_ia += 1
                return
            elif sum(case != 0 for case in self.plateau) == 8:
                self.terminer_partie("Match nul!", None)
                self.parties_nulles += 1
                return
        
        self.tour_joueur = JOUEUR_HUMAIN
    
    def verifier_victoire(self, joueur: int) -> bool:
        """Vérifie si le joueur a gagné et stocke la ligne gagnante"""
        for i in range(3):
            if all(self.plateau[i*3 + j] == joueur for j in range(3)):
                self.ligne_gagnante = [(i*3 + j) for j in range(3)]
                return True
        for j in range(3):
            if all(self.plateau[i*3 + j] == joueur for i in range(3)):
                self.ligne_gagnante = [(i*3 + j) for i in range(3)]
                return True
        if all(self.plateau[i*3 + i] == joueur for i in range(3)):
            self.ligne_gagnante = [(i*3 + i) for i in range(3)]
            return True
        if all(self.plateau[i*3 + (2-i)] == joueur for i in range(3)):
            self.ligne_gagnante = [(i*3 + (2-i)) for i in range(3)]
            return True
        return False
    
    def plateau_plein(self) -> bool:
        """Vérifie si le plateau est plein"""
        return all(case != 0 for case in self.plateau)
    
    def terminer_partie(self, message: str, gagnant: Optional[int]):
        """Termine la partie"""
        self.partie_terminee = True
        self.message_status = message
        self.gagnant = gagnant
        self.animation_victoire = 60
        self.historique_parties.append({
            'date': datetime.now().isoformat(),
            'gagnant': gagnant,
            'plateau': self.plateau.copy(),
            'modele_utilise': self.modele_charge
        })
    
    def nouvelle_partie(self):
        """Démarre une nouvelle partie"""
        self.plateau = [0] * 9
        self.tour_joueur = JOUEUR_HUMAIN
        self.partie_terminee = False
        self.gagnant = None
        self.ligne_gagnante = None
        self.animation_victoire = 0
        self.animation_coup_ia = 0
        self.derniere_position_ia = None
        self.message_status = "Votre tour! Cliquez sur une case." if self.modele_charge else "IA basique - Votre tour!"
    
    def reset_scores(self):
        """Remet les scores à zéro"""
        self.score_humain = 0
        self.score_ia = 0
        self.parties_nulles = 0
        self.historique_parties = []
        self.message_status = "Scores remis à zéro"
    
    def draw_scores(self):
        """Dessine les scores au-dessus du plateau dans des conteneurs individuels"""
        y_scores = BOARD_OFFSET_Y - 100
        container_width = 120
        container_height = 80
        spacing = 20
        
        total_width = 3 * container_width + 2 * spacing
        x_start = BOARD_OFFSET_X + (BOARD_SIZE - total_width) // 2
        
        # Conteneur pour Humain
        humain_rect = pygame.Rect(x_start, y_scores, container_width, container_height)
        pygame.draw.rect(self.screen, LIGHT_GRAY, humain_rect, border_radius=15)
        pygame.draw.rect(self.screen, BLACK, humain_rect, 2, border_radius=15)
        humain_text = self.font_normal.render("Humain", True, RED)
        humain_score_text = self.font_normal.render(str(self.score_humain), True, RED)
        humain_text_rect = humain_text.get_rect(center=(humain_rect.centerx, humain_rect.centery - 10))
        humain_score_rect = humain_score_text.get_rect(center=(humain_rect.centerx, humain_rect.centery + 10))
        self.screen.blit(humain_text, humain_text_rect)
        self.screen.blit(humain_score_text, humain_score_rect)
        
        # Conteneur pour Nuls
        nul_rect = pygame.Rect(x_start + container_width + spacing, y_scores, container_width, container_height)
        pygame.draw.rect(self.screen, LIGHT_GRAY, nul_rect, border_radius=15)
        pygame.draw.rect(self.screen, BLACK, nul_rect, 2, border_radius=15)
        nul_text = self.font_normal.render("Nuls", True, GRAY)
        nul_score_text = self.font_normal.render(str(self.parties_nulles), True, GRAY)
        nul_text_rect = nul_text.get_rect(center=(nul_rect.centerx, nul_rect.centery - 10))
        nul_score_rect = nul_score_text.get_rect(center=(nul_rect.centerx, nul_rect.centery + 10))
        self.screen.blit(nul_text, nul_text_rect)
        self.screen.blit(nul_score_text, nul_score_rect)
        
        # Conteneur pour IA
        ia_rect = pygame.Rect(x_start + 2 * (container_width + spacing), y_scores, container_width, container_height)
        pygame.draw.rect(self.screen, LIGHT_GRAY, ia_rect, border_radius=15)
        pygame.draw.rect(self.screen, BLACK, ia_rect, 2, border_radius=15)
        ia_text = self.font_normal.render("IA", True, BLUE)
        ia_score_text = self.font_normal.render(str(self.score_ia), True, BLUE)
        ia_text_rect = ia_text.get_rect(center=(ia_rect.centerx, ia_rect.centery - 10))
        ia_score_rect = ia_score_text.get_rect(center=(ia_rect.centerx, ia_rect.centery + 10))
        self.screen.blit(ia_text, ia_text_rect)
        self.screen.blit(ia_score_text, ia_score_rect)
    
    def draw_plateau(self):
        """Dessine le plateau de jeu"""
        plateau_rect = pygame.Rect(BOARD_OFFSET_X, BOARD_OFFSET_Y, BOARD_SIZE, BOARD_SIZE)
        pygame.draw.rect(self.screen, WHITE, plateau_rect)
        pygame.draw.rect(self.screen, BLACK, plateau_rect, 3)
        
        for i in range(1, 3):
            x = BOARD_OFFSET_X + i * CELL_SIZE
            pygame.draw.line(self.screen, BLACK, (x, BOARD_OFFSET_Y), (x, BOARD_OFFSET_Y + BOARD_SIZE), 3)
            y = BOARD_OFFSET_Y + i * CELL_SIZE
            pygame.draw.line(self.screen, BLACK, (BOARD_OFFSET_X, y), (BOARD_OFFSET_X + BOARD_SIZE, y), 3)
        
        for i in range(9):
            row, col = i // 3, i % 3
            x = BOARD_OFFSET_X + col * CELL_SIZE + CELL_SIZE // 2
            y = BOARD_OFFSET_Y + row * CELL_SIZE + CELL_SIZE // 2
            if self.plateau[i] == JOUEUR_HUMAIN:
                self.draw_x(x, y, i)
            elif self.plateau[i] == JOUEUR_IA:
                self.draw_o(x, y, i)
    
    def draw_x(self, x: int, y: int, position: int):
        """Dessine un X"""
        size = CELL_SIZE // 3
        color = RED
        pygame.draw.line(self.screen, color, (x - size, y - size), (x + size, y + size), 8)
        pygame.draw.line(self.screen, color, (x - size, y + size), (x + size, y - size), 8)
    
    def draw_o(self, x: int, y: int, position: int):
        """Dessine un O"""
        size = CELL_SIZE // 3
        color = BLUE
        if self.animation_coup_ia > 0 and position == self.derniere_position_ia:
            scale = 1.0 + 0.3 * math.sin(self.animation_coup_ia * 0.3)
            size = int(size * scale)
            color = GOLD
        pygame.draw.circle(self.screen, color, (x, y), size, 8)
    
    def draw_ligne_gagnante(self):
        """Dessine la ligne gagnante avec animation"""
        if not self.ligne_gagnante or self.animation_victoire <= 0:
            return
        
        positions = []
        for pos in self.ligne_gagnante:
            row, col = pos // 3, pos % 3
            x = BOARD_OFFSET_X + col * CELL_SIZE + CELL_SIZE // 2
            y = BOARD_OFFSET_Y + row * CELL_SIZE + CELL_SIZE // 2
            positions.append((x, y))
        
        alpha = int(128 + 127 * math.sin(self.animation_victoire * 0.2))
        color = GOLD if self.gagnant == JOUEUR_IA else GREEN
        if len(positions) >= 2:
            pygame.draw.line(self.screen, color, positions[0], positions[-1], 10)
    
    def draw_result(self):
        """Dessine le résultat sous le plateau"""
        if self.partie_terminee:
            result_text = self.font_normal.render(self.message_status, True, BLACK)
            x_result = BOARD_OFFSET_X + BOARD_SIZE // 2 - result_text.get_width() // 2
            y_result = BOARD_OFFSET_Y + BOARD_SIZE + 20
            self.screen.blit(result_text, (x_result, y_result))

# Lancement du jeu
if __name__ == "__main__":
    jeu = MorpionPygame()
    jeu.run()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\miran\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DummyClassifier from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


❌ Erreur chargement modèle: <class 'numpy.random._mt19937.MT19937'> is not a known BitGenerator module.


SystemExit: 

C:\Users\miran\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
